In [1]:
import os
import glob

import numpy as np
import geopandas as gpd
import pandas as pd
import ee

from soiltexture import getTextures
import matplotlib
from sklearn.metrics import (
    accuracy_score,
    cohen_kappa_score,
    classification_report,
    ConfusionMatrixDisplay,
)

from tqdm.notebook import tqdm
from concurrent.futures import as_completed, ThreadPoolExecutor, ProcessPoolExecutor


## Get Earth Engine Running
To access GEE, we will need to authenticate our account, )&( then initialize a connection to a server. 

In [2]:
SERVICE_ACCOUNT = "refit-fvs@refit-fvs.iam.gserviceaccount.com"
credentials = ee.ServiceAccountCredentials(SERVICE_ACCOUNT, "../gee_key.json")
ee.Initialize(credentials)


# Retrieve Soil Texture Data
For each of the plots in a GeoDataFrame, and each year the imagery are available, we will filter the NASA Soil Moisture Active Passive (SMAP) collection from GEE to our Area of Interest. We want to get a monthly time-series of soil profile moisture for each point.

In [ ]:
DATA_DIR = "../data/"
PLOTS = os.path.join(DATA_DIR, "interim", "plot_info_for_climatena.csv")
plots = pd.read_csv(PLOTS).rename({"ID1": "PLOT_ID"}, axis=1).drop(["ID2"], axis=1)
plots.head()


In [ ]:
plots.info()


## Collection Processing Functions
The following functions work on Google Earth Engine ImageCollections. 

In [ ]:
SOILS = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02")


def get_soil_texture(x, y, buffer=1, epsg=4326):
    aoi = ee.Geometry.Point(x, y, epsg=epsg)
    result = SOILS.reduceRegion(
        reducer=ee.Reducer.mode(),
        geometry=aoi.buffer(buffer),
        crs=f"EPSG:{epsg}",
        scale=1,
    )
    return result.getInfo()


def get_soil_df(point_id, x, y, buffer=1, epsg=4326):
    values = get_soil_texture(x, y, buffer=buffer, epsg=epsg)
    df = pd.DataFrame(
        data=values, dtype=int, index=pd.Series(point_id, name="PLOT_ID", dtype=int)
    )
    while df.isna().values.sum() > 0:
        buffer += 10
        buffer = (buffer // 10) * 10
        values = get_soil_texture(x, y, buffer=buffer, epsg=epsg)
        df = pd.DataFrame(
            data=values, dtype=int, index=pd.Series(point_id, name="PLOT_ID", dtype=int)
        )
    return df


In [ ]:
sand = ee.Image("projects/soilgrids-isric/sand_mean")
silt = ee.Image("projects/soilgrids-isric/silt_mean")
clay = ee.Image("projects/soilgrids-isric/clay_mean")


def get_isric_texture(x, y, buffer=1, epsg=4326):
    aoi = ee.Geometry.Point(x, y, epsg=epsg)
    sand_result = sand.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi.buffer(buffer),
        crs=f"EPSG:{epsg}",
        scale=1,
    )
    silt_result = silt.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi.buffer(buffer),
        crs=f"EPSG:{epsg}",
        scale=1,
    )

    clay_result = clay.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi.buffer(buffer),
        crs=f"EPSG:{epsg}",
        scale=1,
    )
    return {**sand_result.getInfo(), **silt_result.getInfo(), **clay_result.getInfo()}


def get_isric_df(point_id, x, y, buffer=1, epsg=4326):
    values = get_isric_texture(x, y, buffer=buffer, epsg=epsg)
    df = pd.DataFrame(
        data=values, dtype=int, index=pd.Series(point_id, name="PLOT_ID", dtype=int)
    )
    while df.isna().values.sum() > 0 and buffer < 500:
        buffer += 10
        buffer = (buffer // 10) * 10
        values = get_isric_texture(x, y, buffer=buffer, epsg=epsg)
        df = pd.DataFrame(
            data=values, dtype=int, index=pd.Series(point_id, name="PLOT_ID", dtype=int)
        )
    df.columns = [col.replace("_mean", "") for col in df.columns]
    return df


In [ ]:
OVERWRITE = False
results = []

ALREADY_DONE = os.path.join(DATA_DIR, "interim", "soil_types.csv")
if os.path.exists(ALREADY_DONE) and not OVERWRITE:
    already_done = pd.read_csv(ALREADY_DONE).set_index("PLOT_ID")
    results.append(already_done)
    already_done_plots = np.unique(already_done.index.get_level_values(0))
else:
    already_done_plots = []

with ProcessPoolExecutor(10) as executor:
    print("Starting to get data from Google Earth Engine.")
    jobs = [
        executor.submit(get_soil_df, *[row["PLOT_ID"], row["lon"], row["lat"]])
        for _, row in plots.iterrows()
        if row["PLOT_ID"] not in already_done_plots
    ]

    for job in tqdm(as_completed(jobs), total=len(jobs)):
        results.append(job.result())

COL_ORDER = ["b0", "b10", "b30", "b60", "b100", "b200"]
result_df = pd.concat(results, axis=0)[COL_ORDER].dropna()
out_csv = os.path.join(DATA_DIR, "interim", "soil_types.csv")
result_df.to_csv(out_csv, index=True, header=True)
result_df.info()


In [ ]:
OVERWRITE = False
isric_results = []

ALREADY_DONE = os.path.join(DATA_DIR, "interim", "isric_soil_types.csv")
if os.path.exists(ALREADY_DONE) and not OVERWRITE:
    already_done = pd.read_csv(ALREADY_DONE).set_index("PLOT_ID")
    isric_results.append(already_done)
    already_done_plots = np.unique(already_done.index.get_level_values(0))
else:
    already_done_plots = []

with ProcessPoolExecutor(10) as executor:
    print("Starting to get data from Google Earth Engine.")
    jobs = [
        executor.submit(get_isric_df, *[row["PLOT_ID"], row["lon"], row["lat"]])
        for _, row in plots.iterrows()
        if row["PLOT_ID"] not in already_done_plots
    ]

    for job in tqdm(as_completed(jobs), total=len(jobs)):
        isric_results.append(job.result())

isric_result_df = pd.concat(isric_results, axis=0).dropna()

CROSSWALK = {
    "clay": 1,
    "silty clay": 2,
    "silty clay loam": 5,
    "sandy clay": 3,
    "sandy clay loam": 6,
    "clay loam": 4,
    "silt": 10,
    "silt loam": 8,
    "loam": 7,
    "sand": 12,
    "loamy sand": 11,
    "sandy loam": 9,
}
crosswalk = pd.Series(CROSSWALK, name="SOIL_TYPE")

for depth in ["0-5", "5-15", "15-30", "30-60", "60-100", "100-200"]:
    isric_result_df[f"texture_{depth}cm"] = getTextures(
        isric_result_df[f"sand_{depth}cm"].values / 10.0,
        isric_result_df[f"clay_{depth}cm"].values / 10.0,
    )
    isric_result_df[f"soiltype_{depth}cm"] = crosswalk.reindex(
        isric_result_df[f"texture_{depth}cm"].values
    ).values

out_csv = os.path.join(DATA_DIR, "interim", "isric_soil_types.csv")
isric_result_df.to_csv(out_csv, index=True, header=True)
isric_result_df.info()


In [ ]:
compare = pd.merge(
    result_df["b10"],
    isric_result_df["soiltype_15-30cm"],
    left_index=True,
    right_index=True,
    how="inner",
)
compare.info()

In [ ]:
LABELS = {
    1: "C",
    2: "TC",
    3: "SC",
    4: "CL",
    5: "TCL",
    6: "SCL",
    7: "L",
    8: "TL",
    9: "SL",
    10: "T",
    11: "LS",
    12: "S",
}
labels = pd.Series(LABELS, name="SOIL_TYPE")

In [ ]:
disp = ConfusionMatrixDisplay.from_predictions(
    compare["b10"],
    compare["soiltype_15-30cm"],
    normalize="true",  # normed by OpenLandMap preds
    labels=labels.index,
    display_labels=labels.values,
    values_format=",.0%",
)

for child in disp.ax_.get_children():
    if isinstance(child, matplotlib.text.Text):
        child.set_fontsize(7)

disp.ax_.set(xlabel="ISRIC", ylabel="OpenLandMap")

In [ ]:
disp = ConfusionMatrixDisplay.from_predictions(
    compare["b10"],
    compare["soiltype_15-30cm"],
    normalize=None,
    labels=labels.index,
    display_labels=labels.values,
    values_format=",.0f",
)

for child in disp.ax_.get_children():
    if isinstance(child, matplotlib.text.Text):
        child.set_fontsize(7)

disp.ax_.set(xlabel="ISRIC", ylabel="OpenLandMap")

In [ ]:
print("Acc:", accuracy_score(compare["b10"], compare["soiltype_15-30cm"]))
print(
    "Kappa:",
    cohen_kappa_score(compare["b10"], compare["soiltype_15-30cm"], weights="linear"),
)
print(
    classification_report(
        compare["b10"],
        compare["soiltype_15-30cm"],
        labels=labels.index,
        target_names=labels.values,
        zero_division=0,
    )
)